In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef

import matplotlib.pyplot as plt
import seaborn as sns

import src.evaluation
import src.config

In [ ]:
# base_palette = sns.color_palette("tab10", 3)
# expanded_palette = [sns.light_palette(color, n_colors=2, input="rgb").as_hex() for color in base_palette]
# expanded_palette_flat = [color for sublist in expanded_palette for color in sublist]
expanded_palette_flat = ['#1f77b4', '#c47c3d', '#2ca02c', '#2ea9ff', '#ff7f0e', '#45f545']
sns.palplot(expanded_palette_flat)

In [ ]:
expanded_palette_flat = [x for x in sns.color_palette("tab20")]
expanded_palette_flat = [
    (0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
    (1.0, 0.4980392156862745, 0.054901960784313725),
    (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
    '#28b0ff',
    '#ffa951',
    '#3eff43',
    ]
sns.palplot(expanded_palette_flat)

In [ ]:
[x for x in sns.color_palette("tab20")]

In [ ]:
results = pd.read_parquet('./results/df_data.parquet.gzip')

In [ ]:
display(results.head())
display(results.sample(5))
print(results.shape)
print(results.columns.tolist())

In [ ]:
sum([len(x) for x in results['Label'].tolist()]), sum([len(x) for x in results['predicted_label_linear_ALL'].tolist()])

In [ ]:
results[results['Label'].str.contains('O')].shape

In [ ]:
list(src.config.label_encoding.keys())

In [ ]:
np.seterr(divide='ignore', invalid='ignore')

In [ ]:
# results.iloc[3000:3001].apply(lambda x: src.evaluation.evaluate_mcc(targets=list(x['Label']), predictions=list(x['predicted_label_linear_ALL']), labels=expert_labels), axis=1)#.at[0]
# res = results.head(1000).apply(lambda x: src.evaluation.evaluate_mcc(targets=list(x['Label']), predictions=list(x['predicted_label_linear_ALL']), labels=expert_labels), axis=1)
# pd.DataFrame(res.to_dict()).T

In [ ]:
# pd.DataFrame(res.to_dict()).T.S.mean()

In [ ]:
expert_labels = ['S', 'L', 'T', 'O', 'I', 'M']
index_names = ['predicted_label_linear_ALL', 'predicted_label_linear_experts', 'predicted_label_linear_experts_imperfect', 'predicted_label_crf_ALL', 'predicted_label_crf_experts', 'predicted_label_crf_experts_imperfect']
real_names = ['Linear Broad', 'Linear Experts Perfect Gate', 'Linear Experts Imperfect Gate', 'CRF Broad', 'CRF Experts Perfect Gate', 'CRF Experts Imperfect Gate']

mcc_values = {}
for index_name, real_name in zip(index_names, real_names):
    # print(index_name, real_name)
    tmp_mcc_values = src.evaluation.evaluate_mcc(
        targets=list(''.join(results['Label'].tolist())),
        predictions=list(''.join(results[index_name].tolist())),
        labels=expert_labels
    )
    # tmp_mcc_values['real_names'] = real_name
    mcc_values.update({real_name: tmp_mcc_values})

In [ ]:
df_mcc_values = pd.DataFrame(mcc_values).reset_index().rename(columns={'index': 'Label'}).melt(id_vars=['Label'], var_name='Model', value_name='MCC')
df_mcc_values['Error'] = df_mcc_values['MCC'].apply(lambda x: x[1])
df_mcc_values['MCC'] = df_mcc_values['MCC'].apply(lambda x: x[0])
# sns.set(style="whitegrid")
# ax = sns.barplot(data=df_mcc_values, ci=None)
# ax.set(ylim=(0.5, None))

In [ ]:
df_mcc_values['Label'] = df_mcc_values['Label'].map({'S': 'Sec/SPI\nSignal (S)', 'L': 'Sec/SPII\nSignal (L)', 'T': 'Tat/SPI Signal\n(T)', 'O': 'Outer\nRegion (O)', 'I': 'Inner Region (I)', 'M': 'Membrane\nRegion (M)'})


In [ ]:
df_mcc_values

In [ ]:
# for calculations of error bars see plot_2.ipynb

# _errors = pd.DataFrame(df_mcc_values.groupby('Model')['MCC'].std()/np.sqrt(36)).to_dict()['MCC']
# df_mcc_values['Error'] = 
_errors = {
    'CRF Broad': 0.01324,
    'CRF Experts Imperfect Gate': 0.005334,
    'CRF Experts Perfect Gate': 0.0043,
    'Linear Broad': 0.00823,
    'Linear Experts Imperfect Gate': 0.007324,
    'Linear Experts Perfect Gate': 0.00830}
df_mcc_values['Error'] = df_mcc_values.apply(lambda row: _errors[row['Model']], axis=1)

In [ ]:
# df_mcc_values

In [ ]:
src.evaluation.plot_mcc_split_label(df_mcc_values, 'mako')

In [ ]:
expert_labels = ['S', 'L', 'T', 'O', 'I', 'M']
index_names = ['predicted_label_linear_ALL', 'predicted_label_linear_experts', 'predicted_label_linear_experts_imperfect', 'predicted_label_crf_ALL', 'predicted_label_crf_experts', 'predicted_label_crf_experts_imperfect']
real_names = ['Linear Broad', 'Linear Experts Perfect Gate', 'Linear Experts Imperfect Gate', 'CRF Broad', 'CRF Experts Perfect Gate', 'CRF Experts Imperfect Gate']
sp_translations = {'S': 'Sec/SPI Signal (S)', 'L': 'Sec/SPII Signal (L)', 'T': 'Tat/SPI Signal (T)', 'O': 'Outer\nRegion (O)', 'I': 'Inner Region (I)', 'M': 'Membrane\nRegion (M)'}

mcc_values = {}
for index_name, real_name in zip(index_names, real_names):
    # print(index_name, real_name)
    mcc_kingdom_split = {}
    for kindom in results['Kingdom'].unique():
        tmp_mcc_values = src.evaluation.evaluate_mcc(
            targets=list(''.join(results[results['Kingdom'] == kindom]['Label'].tolist())),
            predictions=list(''.join(results[results['Kingdom'] == kindom][index_name].tolist())),
            labels=expert_labels
        )
        # tmp_mcc_values['real_names'] = real_name
        mcc_kingdom_split.update({kindom: tmp_mcc_values})
    mcc_values.update({index_name: mcc_kingdom_split})

In [ ]:
res = []
for _model in index_names:
    melted_model_df = pd.DataFrame(mcc_values[_model]).reset_index().melt(id_vars=['index'], var_name='Category', value_name='Value').rename(columns={'index': 'Type', 'Value': 'MCC', 'Category': 'Kingdom'})
    melted_model_df['Error'] = melted_model_df['MCC'].apply(lambda x: x[1])
    melted_model_df['MCC'] = melted_model_df['MCC'].apply(lambda x: x[0])
    melted_model_df['Kingdom'] = melted_model_df['Kingdom'].apply(lambda x: x.capitalize())
    melted_model_df = melted_model_df[melted_model_df['MCC'] != 0]
    melted_model_df['Model'] = _model
    melted_model_df['Simple_Type'] = melted_model_df['Type']
    melted_model_df['Type'] = melted_model_df['Type'].map(sp_translations)
    melted_model_df['Model'] = melted_model_df['Model'].map(dict(zip(index_names, real_names)))
    melted_model_df
    res.append(melted_model_df)
melted_df = pd.concat(res, axis=0)

In [ ]:
melted_df['Kingdom'].unique()

In [ ]:
_kindom = 'Eukarya'
melted_df[melted_df['Kingdom'] == _kindom]

In [ ]:
# errors_euk = melted_df['MCC'].tolist()

In [ ]:
np.std([1,1,1,1,1,1,1]) 

In [ ]:
# for calculations of error bars see plot_2.ipynb
euk_errors = [0.008843030020910053, 0.00833780700401309, 0.007915217874895991, 0.007316060848095336, 0.009115621915477509, 0.008565877086419958, 0.008689663225525779, 0.007079087855268991, 0.007252493839335378, 0.008858319950672334, 0.007091171023075689, 0.006762150982145675, 0.0126766244754739, 0.011210841833259048, 0.01106541252924351, 0.011279948870277412, 0.012707538771774397, 0.012471136632013272, 0.010360234361080054, 0.01072674961623544, 0.011039117982897168, 0.010161784816472558, 0.012885086757797334, 0.011990696776234289]
arch_errors = [0.0570257749629035, 0.05318576654181419, 0.058655320808664105, 0.05040295880144623, 0.05206111440133385, 0.04468544163653696, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03880812077908939, 0.038795478031595824, 0.04044745004017241, 0.035586916273101316, 0.033628655235951294, 0.03713248856734938, 0.04672237790843924, 0.05306653023420957, 0.0468810202018848, 0.04672756930889681, 0.048639391891150525, 0.0558012620868559, 0.06182629446191362, 0.053450331810326226, 0.05168790503191697, 0.0461437602747725, 0.0469880989060047, 0.057777125800605095, 0.0315797786967633, 0.03463063818632054, 0.032645192378800245, 0.03599003010818213, 0.03908640064739608, 0.03733106280105193]
pos_errors = [0.014958855027037627, 0.016067467971970072, 0.014849292746488017, 0.01695309332213373, 0.015909081589966453, 0.016878190941892177, 0.018196917543079432, 0.019435384876713077, 0.019886977830259337, 0.02131185432994776, 0.021351606816305782, 0.02100927056658151, 0.02319138343910088, 0.022494469809664727, 0.022639048883610062, 0.020901924315945137, 0.021976930611578648, 0.02343826076051575, 0.014572778894672116, 0.015307861947454912, 0.014377802398644298, 0.01480855217635864, 0.014277977093943454, 0.015705847670803713, 0.02520173029719405, 0.023344431434653323, 0.02487655545584469, 0.02496520999738448, 0.022522177356044403, 0.024583532997250235, 0.01669860001117556, 0.014853122910878558, 0.0148635910230332, 0.014668706280056145, 0.015666130244625036, 0.015392455640762606]
neg_errors = [0.016661646056817914, 0.016766958815341952, 0.01574055772553145, 0.014946210472860141, 0.014553584484344436, 0.01665738632577527, 0.016682474194872433, 0.014566882671536231, 0.014731098850789532, 0.01635270767567701, 0.015104564132538517, 0.016476313900138647, 0.014278107041724929, 0.012618984509040471, 0.012242532674407829, 0.012622113923952086, 0.014534694455459045, 0.012957222545289384, 0.014925164883714216, 0.015241106839876083, 0.014232306609487646, 0.013511709071622212, 0.013650596172248704, 0.013437776980133879, 0.01648252753581916, 0.016222883342884787, 0.015680820215758874, 0.01658064992116396, 0.01725076012000547, 0.01570161160260351, 0.017182183509981352, 0.016985630736950797, 0.017035950499616766, 0.01725996860636593, 0.014987814841773453, 0.01688088348741055]

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=2)
sns.set_palette(expanded_palette_flat)
# _palette = expanded_palette_flat
_palette = 'mako'
src.evaluation.plot_mcc_split_label_kingdom_facet_simple(melted_df[melted_df['Kingdom'] == 'Eukarya'], Title='Eukarya', ax=axs[0][0], palette=_palette, errors=euk_errors)
src.evaluation.plot_mcc_split_label_kingdom_facet_simple(melted_df[melted_df['Kingdom'] == 'Archaea'], Title='Archaea', ax=axs[0][1], palette=_palette, errors=arch_errors)
src.evaluation.plot_mcc_split_label_kingdom_facet_simple(melted_df[melted_df['Kingdom'] == 'Positive'], Title='Positive', ax=axs[1][0], palette=_palette, errors=pos_errors)
src.evaluation.plot_mcc_split_label_kingdom_facet_simple(melted_df[melted_df['Kingdom'] == 'Negative'], Title='Negative', ax=axs[1][1], palette=_palette, errors=neg_errors)

# fig.suptitle("Scores for Different Models", fontsize=25)
plt.subplots_adjust(wspace=0.15, hspace=0.25)
sns.set(font_scale=1.6)

plt.legend(loc='upper center', bbox_to_anchor=(-0.1, -0.15), frameon=False, ncol=3, columnspacing=0.7, handletextpad=0.5)


In [ ]:
src.evaluation.plot_mcc_split_label_kingdom_facet(
    melted_df,
    Title=f'{_kindom} - Scores for Different Models'
    )

In [ ]:
# for i, x in enumerate(melted_df['Kingdom'].unique()):
#     axs[i] = src.evaluation.plot_mcc_split_label_kingdom_facet(melted_df[melted_df['Kingdom'] == x], Title=f'{_kindom} - Scores for Different Models')

In [ ]:
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        'display.max_colwidth', 100
#                        ):
#     display(melted_df)

In [ ]:
# df_mcc_values = pd.DataFrame(mcc_values).reset_index().rename(columns={'index': 'Label'}).melt(id_vars=['Label'], var_name='Model', value_name='MCC')
# df_mcc_values

In [ ]:
current = 'predicted_label_linear_experts_imperfect'
# current = 'Label'

# target = list(''.join(results['Label'].tolist()))
# prediction = list(''.join(results[current].tolist()))

# target = list(results['Label'].at[1])
# prediction = list(results[current].at[1])

# print(len(target), len(prediction))
# print(target)
# print(prediction)

# expert_labels = list(src.config.label_encoding.keys())
# expert_labels = list(['S', 'L', 'T' 'O', 'I', 'M'])
expert_labels = ['S', 'L', 'T', 'O', 'I', 'M']
# print(expert_labels)

evaluation_1 = src.evaluation.evaluate_mcc(
    targets=target,
    predictions=prediction,
    labels=expert_labels,
)

# evaluation_2 = src.evaluation.evaluate(
#     targets = target,
#     predictions = prediction,
#     labels = expert_labels,
# )

# print(evaluation_1)
print(*evaluation_1.items(), sep='\n')
# display(src.evaluation.plot_confusion_matrix(evaluation_1['CM'], expert_labels, title=f"Confusion Matrix for {current}"))

In [ ]:
current = 'predicted_label_linear_experts_imperfect'
target = list(''.join(results['Label'].tolist()))
prediction = list(''.join(results[current].tolist()))

expert_labels = ['S', 'L', 'T', 'O', 'I', 'M']
evaluation_1 = src.evaluation.evaluate(
    targets=target,
    predictions=prediction,
    labels=expert_labels,
)
lower_right_3x3 = evaluation_1['CM'][-3:, -3:]
display(src.evaluation.plot_confusion_matrix(lower_right_3x3, labels=expert_labels[-3:], title=f"Confusion Matrix for Linear Experts Imperfect Gate"))

In [ ]:
display(src.evaluation.plot_confusion_matrix(evaluation_1['CM'], labels=expert_labels, title=f"Confusion Matrix for Linear Experts Perfect Gate"))

---
## Sequence Preds

In [ ]:
gate_labels = ['NO_SP', 'SP', 'LIPO', 'TAT']

metrics_gate = src.evaluation.evaluate(
    targets=results['Type'],
    predictions=results['predicted_type'],
    labels=gate_labels
)
print(metrics_gate)
print(metrics_gate['TPR'].mean())
print(metrics_gate['ACC'].mean())
print(metrics_gate['MCC_2'].mean())
# display(src.evaluation.plot_confusion_matrix(metrics_gate['CM'], gate_labels))

In [ ]:
sample_metric = "ACC"
B = 1000
samples = np.empty(B)

for x in range(B):
    sample = results.sample(5)[['Type', 'predicted_type']]
    targets = sample['Type']
    predictions = sample['predicted_type']
    
    # sample_metrics_gate = src.evaluation.evaluate(
    #     targets=targets,
    #     predictions=predictions,
    #     labels=gate_labels
    # )
    
    sample_metrics_gate = np.empty(len(gate_labels))
    for i, item in enumerate(gate_labels):
        target = [1 if y == item else 0 for y in targets]
        prediction = [1 if y == item else 0 for y in predictions]
        if target == prediction:
            sample_metrics_gate[i] = 1
        else:
            sample_metrics_gate[i] = matthews_corrcoef(targets.tolist(), predictions.tolist())
    
    # print(targets.tolist())
    # print(predictions.tolist())
    # print(sample_metrics_gate[sample_metric])
    # print(sample_metrics_gate.mean())
    
    samples[x] = sample_metrics_gate.mean()#np.nanmean(sample_metrics_gate[sample_metric])
display(np.sqrt(1/(B-1)*np.sum((samples-samples.mean())**2)))
# src.evaluation.plot_confusion_matrix(metrics_gate['CM'], gate_labels, title="Confusion Matrix for Gate")

In [ ]:
samples.mean()

---

In [ ]:
# sample = results.sample(5)[['Type', 'predicted_type']]
sample = results[['Type', 'predicted_type']]
targets = sample['Type']
predictions = sample['predicted_type']

In [ ]:
# display(targets)
# display(predictions)

In [ ]:
sample_metrics_gate = src.evaluation.evaluate(
        targets=targets,
        predictions=predictions,
        labels=gate_labels
    )
display(gate_labels)
display(sample_metrics_gate)

In [ ]:
src.evaluation.plot_confusion_matrix(sample_metrics_gate['CM'], gate_labels, title="Confusion Matrix for Gate")